This file contains all definitions needed for the scenario generator.

Please call the 'generate_scenario' function to get a new, random scenario. Note you can enter a custom random seed.

Please note that scenarios are generated using the following logic:
1. Under the 'Default values' markdown, you can see the expected number of passengers for evening rush hour, for any given workday.
2. This expected count across two hours is broken down into all of its five minute intervals (24 intervals total)
3. If E is the expected count for any line, we then have an expected E/24 count for each interval.
4. We could then generate a count for each inteval and line, so that it follows a Normal distribution with mean E/24 and std. dev 0.15*(E/24)
5. Since Normal could return negative, get max(count, 0), meaning nobody showed up to take the line (should happen rarely)
6. Then, we floor the returned value, as Normal can return non integer.

In [114]:
import numpy as np
from typing import Dict, Tuple
from time import time_ns

Default values

In [115]:
EXP_RIDERSHIP: Dict[str, int] = {
    'Lakeshore West': 33239,
    'Lakeshore East': 25769,
    'Kitchener': 15071,
    'Milton': 13138,
    'Barrie': 8636,
    'Stouffville': 7341,
    'Richmond Hill': 4642
}

SPREAD = 0.15 #Modify this to get scenarios further from the exp values above

Function for scenarios

In [116]:
def generate_scenario(seed: int | str = None) -> Tuple[Dict[str, Dict[str, int]], Dict[str, int]]:
    """
    Generates ridership demand scenario.

    Returns dict of all time interval and line combinations, and dict of summed up generated demands for each line.

    seed: int | str => can be used for replication purposes. Random seed to use.
    """

    if seed is None: seed = time_ns()%(2**32-1)

    if type(seed) is not int: seed = sum([ord(char) for char in seed])

    np.random.seed(seed)

    scenario: Dict[str, Dict[str, int]] = {}

    #Initializing dicts
    for line in EXP_RIDERSHIP.keys():
        scenario[line] = {}

    #Iterating lines
    for line, exp_demand in EXP_RIDERSHIP.items():
        mu = exp_demand/24
        sigma = mu*SPREAD

        #Iterating time intervals
        for t in range(0, 120, 5):
            demand = np.random.normal(loc=mu, scale=sigma)
            scenario[line][f"{4+int(t/60)}:{(t-60*int(t/60)):02}"] = max(0, int(demand))

    #Total demand
    total_demands: Dict[str, int] = {}
    for lines_, demands in scenario.items():
        total_demands[lines_] = sum(demands.values())

    return scenario, total_demands



In [117]:
generate_scenario("MIE562")

({'Lakeshore West': {'4:00': 1656,
   '4:05': 1297,
   '4:10': 1178,
   '4:15': 1631,
   '4:20': 1465,
   '4:25': 1700,
   '4:30': 1175,
   '4:35': 1654,
   '4:40': 1449,
   '4:45': 1522,
   '4:50': 1440,
   '4:55': 1399,
   '5:00': 1540,
   '5:05': 1236,
   '5:10': 1408,
   '5:15': 1102,
   '5:20': 1554,
   '5:25': 1396,
   '5:30': 1500,
   '5:35': 1399,
   '5:40': 1424,
   '5:45': 1879,
   '5:50': 1153,
   '5:55': 1543},
  'Lakeshore East': {'4:00': 1267,
   '4:05': 1111,
   '4:10': 1098,
   '4:15': 1400,
   '4:20': 832,
   '4:25': 1018,
   '4:30': 701,
   '4:35': 1232,
   '4:40': 955,
   '4:45': 1302,
   '4:50': 920,
   '4:55': 1276,
   '5:00': 814,
   '5:05': 1052,
   '5:10': 1126,
   '5:15': 1074,
   '5:20': 911,
   '5:25': 1116,
   '5:30': 1100,
   '5:35': 1040,
   '5:40': 971,
   '5:45': 1315,
   '5:50': 827,
   '5:55': 870},
  'Kitchener': {'4:00': 657,
   '4:05': 607,
   '4:10': 471,
   '4:15': 655,
   '4:20': 620,
   '4:25': 604,
   '4:30': 594,
   '4:35': 798,
   '4:40': 732

In [139]:
generate_scenario()

({'Lakeshore West': {'4:00': 1626,
   '4:05': 1252,
   '4:10': 1314,
   '4:15': 1117,
   '4:20': 1548,
   '4:25': 1716,
   '4:30': 1366,
   '4:35': 1067,
   '4:40': 1386,
   '4:45': 1673,
   '4:50': 1572,
   '4:55': 1344,
   '5:00': 1318,
   '5:05': 1500,
   '5:10': 1255,
   '5:15': 1431,
   '5:20': 1396,
   '5:25': 1431,
   '5:30': 1187,
   '5:35': 1353,
   '5:40': 1767,
   '5:45': 1326,
   '5:50': 1038,
   '5:55': 1149},
  'Lakeshore East': {'4:00': 1065,
   '4:05': 661,
   '4:10': 962,
   '4:15': 1116,
   '4:20': 944,
   '4:25': 1240,
   '4:30': 1076,
   '4:35': 976,
   '4:40': 889,
   '4:45': 1010,
   '4:50': 895,
   '4:55': 1128,
   '5:00': 719,
   '5:05': 971,
   '5:10': 1168,
   '5:15': 1184,
   '5:20': 1133,
   '5:25': 992,
   '5:30': 904,
   '5:35': 984,
   '5:40': 1115,
   '5:45': 1045,
   '5:50': 1093,
   '5:55': 1030},
  'Kitchener': {'4:00': 612,
   '4:05': 680,
   '4:10': 502,
   '4:15': 687,
   '4:20': 754,
   '4:25': 642,
   '4:30': 645,
   '4:35': 550,
   '4:40': 597,
